# Abstract

**Huffman Encoding** is one of the most basic and elegant applications of the **Greedy Algorithm Design Paradigm**. It provides an optimal method of **lossless data compression** by assigning shorter binary codes to frequently occurring symbols and longer codes to rarely occurring ones.
This project implements Huffman Encoding and Decoding in **Python 3.14.0**, constructs frequency tables and Huffman Trees, and evaluates performance through compression ratio analysis.  
The project presents both the **proof of correctness** and **complexity analysis** of the algorithm and compares the bit cost with that of **fixed-length encoding**.
Experimental results show that Huffman encoding achieves **significant space reduction** when symbol distributions are **non-uniform**, validating its theoretical optimality in practice.


# 1. Introduction and Motivation

In the digital era, where vast quantities of data are produced every second, efficient storage and transmission have become crucial. **Data compression** seeks to represent information using fewer bits than the original form, thereby saving both storage space and communication bandwidth.

Among several compression methods, **Huffman Encoding** stands out as a classic example of a **Greedy Algorithm** that achieves **lossless compression**. It assigns variable-length binary codes to symbols such that frequently occurring symbols receive shorter codes, while rare symbols are given longer codes. But this needs to be done carefully as it is susceptible to ambiguous interpretation.To emphasize this we consider the example of the Morse code,


Traditionally, according to the number of unique characters, we assign a fixed-length bit string to represent each symbol, as done in encoding schemes such as ASCII. However, characters in real-world data do not occur uniformly—some appear far more frequently than others. To save space, the characters that occur more often should be assigned shorter bit strings, while those that occur less frequently can be assigned longer ones. The challenge, therefore, is to determine an optimal way of assigning these variable-length codes so as to minimize the overall number of bits required.

Morse code, in a way, follows a similar intuition by assigning simpler symbols or shorter actions to frequently occurring letters. However, it does not satisfy the **prefix-free property**; instead, it relies on fixed time intervals to mark the beginning and end of each letter.

**Huffman Encoding** effectively addresses this limitation by constructing a prefix-free variable-length code that minimizes the total expected code length, thereby achieving optimal lossless compression.

---

## **Objectives**
The goals of this project are to:
1. Implement Huffman Encoding and Decoding in Python.  
2. Construct frequency tables and Huffman Trees for textual data.  
3. Compute and analyze **compression ratios** against fixed-length encoding.  
4. Verify the **prefix-free property** and correctness of the algorithm.  
5. Analyze time and space complexities and demonstrate optimality through experiment.

---

## **Data Compression and Information Theory**
**Data compression** can be broadly classified as:
- **Lossless Compression:** The original data is perfectly reconstructed after decompression (e.g., Huffman Coding).  
- **Lossy Compression:** Some information is irreversibly discarded to achieve higher compression (e.g., JPEG, MP3).

We focus on the Huffman algorithm, which minimizes the expected number of bits per symbol. The theoretical foundation behind the **Huffman Algorithm (1952)** lies in **Claude Shannon’s Information Theory (1948)**, where the entropy of a source, given by:

$$
H(X) = -\sum_{i=1}^{n} p_i \log_2 p_i
$$

represents the lower bound on the average number of bits required to encode a message. Huffman’s algorithm constructs a code whose average length \(L\) satisfies:

$$
H(X) \leq L < H(X) + 1
$$

showing that it is asymptotically optimal among all prefix-free codes.

---

### **Why Huffman Algorithm is a Greedy Algorithm**

The **Huffman Algorithm** is a *greedy algorithm* as it satisfies both the **greedy-choice property** and the **optimal substructure property**.  
It works by making the best decision at each step and then proceeding to the next, again choosing the best possible option at that moment — that is, it makes **locally optimal choices** at every stage.

- **Greedy-choice property:** A global optimum can be achieved by choosing a local optimum at each step.  
- **Optimal substructure:** An optimal solution to the problem contains optimal solutions to its subproblems.

Together, these ensure that Huffman’s locally optimal merges of the smallest frequencies lead to a globally optimal prefix-free code.


## **Applications**
Huffman Encoding (or, more broadly, the idea behind it) has numerous applications:
- **File Compression:** Used in ZIP, GZIP, and DEFLATE formats for efficient lossless compression.  
- **Image Compression:** Serves as the entropy coding stage in JPEG.  
- **Audio Compression:** Applied in MP3 and AAC to encode frequency coefficients efficiently.  
- **Data Transmission:** Reduces bandwidth usage in communication systems and embedded devices.  

# **2. Problem Definition and Objectives**

---

# **Problem Statement**

---

## **Formalized Description**

**Input:**

- Alphabet \( A = (a_1, a_2, \dots, a_n) \), which is the symbol alphabet of size \( n \).  
- Tuple \( W = (w_1, w_2, \dots, w_n) \), which is the tuple of positive symbol weights (usually proportional to probabilities), i.e.  
  \( w_i = \text{weight}(a_i), \quad i \in \{1, 2, \dots, n\} \).

**Output:**

- Code $ C(W) = (c_1, c_2, \dots, c_n) $, which is the tuple of binary codewords, where \( c_i \) is the codeword assigned to \( a_i \), 
  for \( i \in \{1, 2, \dots, n\} \).

**Goal:**

Let  

\[
L(C(W)) = \sum_{i=1}^{n} w_i \times \text{length}(c_i)
\]

be the weighted path length of code \( C \).  
The objective is to minimize \( L(C(W)) \), satisfying the condition  

\[
L(C(W)) \leq L(T(W))
\]

for any other code \( T(W) \).

---

## **Informal Description**

**Given:**  
A set of symbols \( S \), and for each symbol \( x \in S \), the frequency \( f_x \) representing the fraction of symbols in the text that are equal to \( x \).

**Find:**  
A prefix-free binary code (a set of codewords) with minimum expected codeword length — equivalently, a binary tree with minimum weighted path length from the root.

---

*Source: Adapted and summarized from the [“Huffman coding” article on Wikipedia](https://en.wikipedia.org/wiki/Huffman_coding).*


## **Input and Output Specification**

- **Input:** A text file or string containing characters.  
- **Output:**
  1. Frequency table and Huffman tree visualization.  
  2. Encoded binary sequence.  
  3. Decoded text identical to input.  
  4. Compression statistics (ratio, space saved).

---

## **Assumptions and Constraints**

- Each input character has a non-negative frequency.  
- All symbols are independent and identically distributed within the input.  
- Encoding must remain prefix-free for correctness.

---

## **Design Goals**

1. Maintain correctness and unique decodability.  
2. Ensure minimal average codeword length.  
3. Optimize both runtime and memory usage.  
4. Compare results with fixed-length encoding using \( \lceil \log_2 n \rceil \) bits per character.


# **3. Algorithm Design**

## **Algorithmic Overview**

The Huffman algorithm is **greedy** in nature — it always tries to obtain the most optimal outcome at each step without considering future consequences.  
It does so by constructing a **prefix-free binary tree**, built iteratively by combining the two least frequent nodes into a single new node and assigning the two original nodes as its left and right children.

In other words, a queue (or priority list) of nodes is maintained, from which the two nodes with the smallest frequencies are repeatedly removed, merged into a new combined node, and then reinserted into the queue. This process continues until only one node remains, which becomes the root of the Huffman tree.

The algorithm can be divided into the following stages:
1. Build a **frequency table** for all unique characters in the text.  
2. Create a **min-heap (priority queue)** of nodes sorted by frequency.  
3. Repeatedly remove the two nodes with the smallest frequencies and merge them into a new node whose frequency is their sum.  
4. Insert the new node back into the heap until only one node remains — the root of the Huffman Tree.  
5. Traverse the tree to assign binary codes: left edge adds a `0` and right edge adds a `1`.

---

---

## **Data Structures Used**
- **Priority Queue (Min-Heap):** Ensures efficient extraction of nodes with smallest frequencies, implemented via Python’s `heapq`.  
- **Binary Tree:** Represents hierarchical structure of merged nodes.  
- **Dictionary:** Stores mapping from characters to their Huffman codes.  

---
## **Pseudocode**

The following pseudocode describes the construction of the Huffman Tree and the generation of the Huffman Codes.

---

### **Algorithm 1: Build Huffman Tree**

Input: Set of symbols C = {c1, c2, ..., cn} with corresponding frequencies f(c)
Output: Root node of the Huffman Tree
```
1. Create a min-heap Q and insert all characters with their frequencies.
2. while size(Q) > 1 do
3. x ← Extract-Min(Q)          // Node with smallest frequency
4. y ← Extract-Min(Q)          // Node with second smallest frequency
5. z ← New node with frequency f(z) = f(x) + f(y)
6. z.left ← x
7. z.right ← y
8. Insert(Q, z)
9. end while
10. return Extract-Min(Q) // The remaining node is the root of the Huffman Tree
 ```
---

### **Algorithm 2: Generate Codes**

Input: Root node of Huffman Tree, current code = ""
Output: Code dictionary for each symbol
```
1. if node is leaf then
2. Assign current code to symbol(node)
3. else
4. GenerateCodes(node.left, code + "0")
5. GenerateCodes(node.right, code + "1")
6. end if
```
---

### **Explanation**

- The **Build Huffman Tree** algorithm constructs the binary tree by repeatedly combining the two least frequent nodes into a new parent node, whose frequency equals their sum.  
- The **Generate Codes** algorithm traverses the final tree recursively to assign binary codes:
  - A left edge adds a `0`
  - A right edge adds a `1`
- The resulting codes are **prefix-free**, meaning no codeword is a prefix of another — ensuring unique and unambiguous decoding.














## **Example: "abracadabra"**

Consider the input string `"abracadabra"`. The frequencies of the characters are:

| Character | Frequency |
|:----------:|:----------:|
| a | 5 |
| b | 2 |
| r | 2 |
| c | 1 |
| d | 1 |

The Huffman algorithm proceeds as follows:
- Merge **c(1)** and **d(1)** → new node with frequency **2**  
- Merge **b(2)** and **r(2)** → new node with frequency **4**  
- Merge node(2) [from c,d] with node(4) [from b,r] → new node with frequency **6**  
- Merge **a(5)** and node(6) → root node with frequency **11**

**Resulting Huffman Codes:**

| Character | Huffman Code |
|:----------:|:-------------:|
| a | 0 |
| r | 100 |
| b | 101 |
| c | 110 |
| d | 111 |

---

## **Encoded Output Length**

The length of the Huffman code for the string `"abracadabra"` is: 01011000110011101011000  which consists of **23 bits** in total.  

In contrast, the fixed-length encoding version would require:

$$
(\text{length of "abracadabra"}) \times \lceil \log_2(5) \rceil = 11 \times 3 = 33
$$

Hence, the fixed-length encoded version would be **33 bits**, while the Huffman-encoded version uses **23 bits**, giving a savings of **10 bits (~30.3% reduction)**.








# **4. Correctness**

---

## **Key Definitions**

**Definition (Prefix-Free Code):**  
A set of binary codewords is prefix-free if no codeword is a prefix of another.

**Definition (Expected Code Length):**  
For probabilities $p_i$ and code lengths $\ell_i$:
$$
L(C) = \sum_i p_i \ell_i
$$
The goal is to minimize $L(C)$.

**Definition (Full Binary Tree Representation):**  
Each prefix-free code corresponds to a full binary tree whose leaves represent the symbols.

---

## **Why the Greedy Step Is Valid**

**Lemma (Sibling Property):**  
In an optimal prefix-free code, the two least probable symbols are siblings at the greatest depth.

**Proof:**  
If not, suppose two other nodes $x', y'$ are deeper. Swapping their positions with the smallest two ($x, y$) can only reduce or preserve the total expected length.  
Thus, we can always arrange for the smallest probabilities to appear as siblings.  

This justifies merging the two smallest weights first in each step.

---

## **Recursive Optimality**

**Theorem (Optimal Substructure):**  
After merging $x$ and $y$ (the smallest nodes) into $z$, finding the optimal code for the reduced set and then expanding $z$ back gives the optimal code for the full set.

**Proof:**  
Let the merged probability be $p' = p_x + p_y$.  
If $L'$ is the minimal length for the smaller instance, expanding $p'$ back adds one bit to each of $x$ and $y$, giving:
$$
L = L' + (p_x + p_y)
$$
Any better tree would imply a smaller $L'$, contradicting optimality.

---

## **Proof of Correctness**

**Theorem (Correctness of Huffman Coding):**  
Huffman’s algorithm produces a prefix-free code of minimum expected length.

**Proof (By Induction on $n$):**  
**Base Case:** For $n = 2$, only one valid prefix-free code exists.  
**Inductive Step:** Assume the result holds for $n-1$ symbols.  
For $n$ symbols, merge the smallest two into one symbol of weight $p'$.  
By induction, the reduced tree is optimal; expanding $p'$ back preserves optimality.

---

## **Intuition Behind Huffman Encoding**

The idea behind Huffman encoding is to assign probabilities to the occurrences of each character in the text and then define $L$ as the total length of the encoded string.  
Ideally, we want to use as little space as possible to represent data. Hence, the algorithm attempts to minimize the **expected value of the code length**, i.e., the average number of bits required to represent a character.

In other words, the algorithm aims to construct a variable-length binary code that minimizes the expected length $E[L]$ of the encoded message.  
To achieve this, characters that occur more frequently are placed **closer to the root** of the Huffman tree (thus having shorter codes), while less frequent characters are placed **deeper in the tree** (thus having longer codes).

This strategy ensures that, on average, the total number of bits used to represent a message is minimized.

---

## **Information-Theoretic Perspective**

From an algorithmic point of view, Huffman encoding minimizes the expected code length $E[L]$ — the average number of bits required to represent a symbol based on its probability of occurrence.  
However, from an **information-theoretic perspective**, the problem can be viewed through the lens of **Shannon’s Entropy**.

Entropy, introduced by Claude Shannon in 1948, measures the fundamental limit of compressibility of any information source. It is defined as:
$$
H(X) = -\sum_{i=1}^{n} p_i \log_2 p_i
$$
where $p_i$ denotes the probability of the $i^{th}$ symbol.

Entropy represents the theoretical lower bound on the average number of bits needed to encode one symbol of the source.  
In other words, no lossless compression algorithm can achieve an average code length smaller than $H(X)$.

Huffman encoding, though derived from a greedy algorithmic process, turns out to be nearly optimal in the information-theoretic sense.  
Its average code length $L$ satisfies the inequality:
$$
H(X) \leq L < H(X) + 1
$$
This means that Huffman coding operates within one bit of the optimal limit imposed by entropy.

Thus, while the algorithm was originally formulated from a procedural optimization viewpoint — minimizing expected code length — it naturally aligns with the deeper theoretical framework of information theory, showing that the greedy construction achieves almost the same efficiency as the best possible code predicted by Shannon’s theory.

---

In practical applications, when probabilities are estimated accurately, Huffman encoding approaches the entropy bound closely.  
When probabilities vary with time or are unknown in advance, extensions such as **adaptive Huffman coding** and **arithmetic coding** are used to maintain or surpass this efficiency.



# **5. Complexity Analysis**

When we implemented the algorithm, we wanted to go beyond just quoting  
the $O(n \log n)$ result from class — to see how that complexity actually arises  
and how the constant factors matter in practice.

---

## **Counting the Work**

Let $n$ be the number of distinct symbols.  
The Huffman process performs $n-1$ merges.  
Each merge removes two smallest elements and inserts one combined node.  
If we store nodes in a **min-heap**, then:

- **extract-min:** $O(\log n)$  
- **insert:** $O(\log n)$

Since each merge performs two removals and one insertion,  
the total work is proportional to $3(n-1)\log n$, i.e.

$$
T(n) = O(n \log n)
$$

---

## **Why This Bound Makes Sense**

The $\log n$ factor comes from maintaining heap order.  
If we used an unsorted list, each merge would require scanning all elements,  
costing $O(n)$ per step — an $O(n^2)$ algorithm overall.  
The heap avoids this by keeping the smallest elements near the top.

---

## **A Quicker Variant (If Already Sorted)**

When input frequencies are sorted, we can build the same Huffman tree  
in linear time using two queues — one for original symbols, one for merged nodes.  
Each node is enqueued and dequeued once, giving $O(n)$ total time.  
But for unsorted data, $O(n \log n)$ is asymptotically optimal.

---

## **Amortized Reasoning**

Every node enters and exits the heap exactly once.  
Dividing the total work evenly, each node costs about $O(\log n)$  
time on average.  
This shows the “amortized” perspective — average cost per operation,  
not worst case per step.

---

## **A More Concrete Cost Picture**

To visualize the cost, each merge step involves two `extract-min`  
and one `insert`, each taking about $\log n$ time.  
Hence over $n-1$ merges,

$$
T(n) \approx 3(n-1)\log n + O(n)
$$

This explains why $O(n \log n)$ is tight and why the heap is ideal here.

---

## **Encoding and Decoding**

Once the tree is ready:

- **Encoding:** $O(M)$ time, where $M$ is message length — we simply look up codes and append bits.  
- **Decoding:** also $O(M)$, traversing one bit at a time from root to leaves.

Both phases are practically linear in message size.

---

## **Space Complexity**

Required data structures:

- Heap (≤ $n$ elements)  
- Huffman tree ($2n-1$ nodes)  
- Code lookup table (1 entry per symbol)

Hence $O(n)$ total space.

---

## **Complexity Summary**

| **Phase**           | **Time**         | **Space** |
|---------------------|------------------|-----------|
| Tree construction   | $O(n \log n)$    | $O(n)$    |
| Encoding            | $O(M)$           | $O(n)$    |
| Decoding            | $O(M)$           | $O(n)$    |

---

In practice, runtime increased roughly as $n \log n$, matching theory.



# **6. Implementation and Experimental Results**

---

## **Programming Environment**
This project was implemented in **Python 3.14.0** using standard library modules only.  
The following libraries were used:
- `heapq` — for implementing the min-heap (priority queue) used in tree construction.  
- `collections.Counter` — for counting symbol frequencies.  
- `math` — for logarithmic calculations in compression metrics.  
- `os` and `sys` — for basic file handling and program control.

All code was written and tested in a **Jupyter Notebook** and a standard **Python environment** (VS Code).  
The implementation focuses on readability and simplicity, with detailed comments explaining every step of the Huffman Encoding and Decoding process.

---

## **Input and Output Interface**
The program allows two ways to provide input:
1. **Manual Text Input:**  
   The user can enter a string directly into the console or notebook cell.
2. **File Input:**  
   A text file can be read from the system for encoding.

The output includes:
- A **frequency table** of characters.  
- A **Huffman tree** constructed from those frequencies.  
- The **encoded bitstring** (saved as `encoded_output.txt`).  
- The **decoded text** (saved as `decoded_output.txt`).  
- Compression statistics such as total bits, fixed-length bits, and percentage of space saved.

---



# **7. Experiments, Datasets and Observations**

---

## **Experimental Datasets**


---

## **Sample Run**


---

## **Compression Analysis**


---

## **Verification**
The decoded output perfectly matched the original input in every test, confirming that the algorithm works correctly.  
The amount of space saved depended on how uneven the character frequencies were — when all characters appeared with similar frequency, compression was minimal, but when certain characters appeared much more often than others, the algorithm achieved a significant reduction in size.

---



# **8. Potential Issues**

Although the Huffman Encoding implementation works well for moderate inputs, a few refinements can make it more robust.

---

## **Memory Management and Large File Handling**

The current implementation loads the entire file into memory before computing frequencies and building the tree.  
For large datasets, this is inefficient and may exceed memory limits.

**Improvement:** Use a *streaming approach* — read data in small chunks and update frequency counts incrementally.  
This allows handling larger files efficiently without exhausting memory.

---

## ** Binary Output Format: Bits vs ASCII Representation**

Encoded data are currently stored as ASCII characters (`'0'` and `'1'`), wasting space since each bit uses an entire byte.

**Improvement:** Output should be written as a true *bitstream*.  
Python libraries such as `bitarray` or `io.BytesIO` can pack bits into bytes efficiently.  
During decoding, bytes can be unpacked bit by bit for traversal, ensuring that compression gains accurately reflect theoretical efficiency.

---

## ** Visualization and Interpretability**

ASCII trees are easy to inspect for small examples but become unreadable for larger datasets.

**Improvement:** Use visualization tools such as `Graphviz` or `NetworkX` to render trees graphically.  
Color-coding leaves (symbols) and internal nodes (merged frequencies) would make the construction process more interpretable and pedagogically useful.

---

## ** Error Handling and Robustness**

The current decoder assumes perfect input; a single corrupted bit may cause complete decoding failure.

**Improvement:** Introduce input validation and fault tolerance.  
Adding checksums or structured exception handling can help detect corrupted files and prevent runtime errors, making the implementation more reliable.

---


# **9. Challenges Faced and Conclusion**

---

## **Challenges Faced**

1. One of the main challenges we faced was setting up the programming environment. Installing Python, configuring the right libraries, and getting Jupyter and Git Bash to work properly took much longer than expected.

2. Even though we understood the Huffman algorithm conceptually, turning it into working code was far from straightforward. It took time to make the computer do exactly what we wanted — especially while handling text files, file paths, and encoding issues. Debugging small mistakes like queue ordering or missing cases often took hours.

<!-- 3. What surprised us most was how much effort it takes to correctly implement something that seems so basic on paper. Huffman’s algorithm is a simple algorithm, but even small modifications, misunderstandings, or blind spots make it tricky to get right in practice. -->

In the end, the experience made us appreciate how theory and implementation are two very different skills — and how much precision and patience real-world coding actually requires.

---

## **Improvements**

Generalizations and future enhancements include:

- Implementing binary-level compression to measure true disk savings.  
- Developing adaptive Huffman and arithmetic coding variants.  
- Applying Huffman coding to multimedia (images, audio).  
- Creating a graphical interface to visualize the encoding process.  
- Integrating entropy-based analysis to compare performance with the Shannon limit.  

---

## **Conclusion**

This project successfully implemented and analyzed **Huffman Encoding**, demonstrating its role as a classic example of a **greedy optimization algorithm**.  
The results verified that Huffman’s approach minimizes the expected number of bits per symbol while preserving exact reconstructability.  

The algorithm was found to be:

- **Correct:** Decoding perfectly reconstructs the original text.  
- **Efficient:** Exhibits near-linear runtime for practical datasets.  
- **Optimal:** Achieves minimal weighted path length among all prefix-free codes.  


---


# **Appendix**

---

## **A. Complete Python Code (Final Submission Version)**






---

## **B. How to Use the Code**
